In [40]:
import numpy as np

# The Lennard Jones Potential 

## Exercises

The Lennard Jones potential is among the simplest analytical potentials, it is a 
two-body potential where the interaction between a pair of atoms is given by
$$
V(r) = 4 \epsilon \left[ \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^{6} \right]
$$

Where $r$ is the distance between the atoms $\sigma$ is a parameter that determines the 
location of the minimum and $\epsilon$ determines the depth of the minimum.

### Exercise: Energy of a pair of atoms.

Implement a function that takes the distance `r`, `sigma` and `epsilon` and returns the Lennard Jones 
potential.

In [1]:
def get_lj_energy(r, sigma, epsilon):
    E = 0 # calculate the Lennard-Jones energy
    return E

It is often best to work with functions that take arrays as that allows for avoiding 
slow for-loops. So test whether your function works with an `numpy` array - otherwise 
change it so that it does;

In [42]:
get_lj_energy(np.array([1, 2, 3]), 1, 1)

0

### Exercise: Energy of an `Atoms` object.

Implement a function `calculate` that takes an `Atoms`-object, `sigma` and `epsilon` and returns the 
total energy according to the Lennard Jones expression. 

Hint: First create an array that contains the distance between all pairs of atoms, 
you can use the [`pdist`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html)
function from scipy to do this. Then pass that to your `get_lj_energy` function and sum 
all the terms together. 

In [ ]:
def calculate(atoms, sigma, epsilon):
    d = 0 # Find the distances between all atoms
    E = 0 # Calculate the Lennard-Jones energy
    return E

### Exercise: Lennard Jones with Object Oriented Programming

The code we have written so far is functional; we have created functions that take some input and return output - without 
any type of state/memory. Yet, the inputs we have passed are not really of the same type - the `Atoms` object is 
really the variable and `sigma` and `epsilon` are parameters of the function. 

If is often beneficial to create classes in these types of situations and is very 
commonly done. 

In [2]:
class LennardJones:

    def __init__(self, sigma, epsilon):
        self.sigma = sigma
        self.epsilon = epsilon
    
    def get_lj_energy(self, r):
        # Calculate the Lennard-Jones energy using the class attributes 
        # self.sigma and self.epsilon. 
        pass

    def calculate(self, atoms):
        # Same logic as before, but now we use the class method get_lj_energy.
        pass

### Exercise: Computing forces

It is often useful to calculate not only the energy but also the force on each atom. The components 
of the forces are given by
$$
F_i^\alpha = -\frac{\partial E}{\partial x_i^\alpha}
$$

Where $i$ indicates the atom and $\alpha$ denotes either the $x$, $y$ or $z$ dimension. 

This can be applied to the Lennard Jones expression, but it is cumbersome and 
more difficult to implement than the potential itself - but if you want to try 
you can. Note that the potential is not given directly in terms of the coordinates, 
but rather in terms of the distance between atoms, so that needs to be taken into account.

This exercise is **optional** and probably best left as an extra for the interested. 

### Lennard Jones with Pytorch

As stepping stone to implementing neural network based potentials we will first 
look at implementing the Lennard Jones potential using Pytorch.

To do so, we should replace all `numpy` or `scipy` functions in our implementation 
with `torch` functions and any use of arrays from `numpy` should be replaced 
with `torch.tensor`.

One of the main benefits of doing this is that we can calculate forces using 
*automatic differentiation* the same technique that enables training of neural networks. 

In the cell below you should fill out the left out code. 

In [85]:
import torch

class LennardJonesTorch:

    def __init__(self, sigma=1.0, epsilon=1.0):
        self.sigma = sigma
        self.epsilon = epsilon

    def atoms_to_tensor(self, atoms):
        return torch.tensor(atoms.positions, requires_grad=True)
    
    def get_lj_energy(self, r):
        # Calculate the Lennard-Jones energy using the class attributes 
        # self.sigma and self.epsilon.
        r6 = r**6
        r12 = r6**2
        return 4 * self.epsilon * (self.sigma/r12 - self.sigma/r6)
    
    def get_forces(self, energy, positions):
        return -torch.autograd.grad(energy, positions, retain_graph=True)[0]

    def calculate(self, atoms):
        # Convert the positions to a tensor
        positions = self.atoms_to_tensor(atoms)
        return self.forward(positions)

    def forward(self, positions):
        # Calculate the pairwise distances
        d = torch.pdist(positions)

        # Calculate the Lennard-Jones energy
        E = torch.sum(self.get_lj_energy(d))
        F = self.get_forces(E, positions)

        # Return the energy and forces
        E = E.detach().numpy() # Convert back to numpy
        F = F.detach().numpy() 
        return E, F

In [86]:
from ase import Atoms

LJT = LennardJonesTorch()

atoms = Atoms('H2', positions=[[0.0, 0.0, 0.0], [1.105, 0.0, 0.0]])

energy, forces = LJT.calculate(atoms)

### Exercise: Using atomic forces

One common use case of atomic forces is local optimization or 'relaxation'. 
Here the energy is minimized by moving the atoms according to the forces, 
this is analagous to a ball rolling down a hill to find a 
position that minimizes the gravitational potential energy. 

ASE makes this very easy, the code below converts your `LennardJonesTorch` 
to an `ase` `Calculator` such that it can interface with the rest of the functionality. 

In [87]:
from imlms import get_calculator_from_class

LJT = get_calculator_from_class(LennardJonesTorch)(sigma=1.0, epsilon=1.0)

atoms = Atoms('H2', positions=[[0.0, 0.0, 0.0], [1.105, 0.0, 0.0]])
atoms.calc = LJT
E = atoms.get_potential_energy()
print(E)

-0.9902696910139173


Read this section on [optimizers](https://wiki.fysik.dtu.dk/ase/gettingstarted/tut02_h2o_structure/h2o.html#optimizers) 
from the `ase` documentation and relax a small molecule, e.g. H2 with your Lennard Jones potential.

In [89]:
from ase.optimize import BFGS

atoms = Atoms('H2', positions=[[0.0, 0.0, 0.0], [1.5, 0.0, 0.0]])
atoms.calc = LJT

optimizer = BFGS(atoms, trajectory='H2.traj')
optimizer.run(fmax=1e-3)

print(atoms.get_potential_energy())

      Step     Time          Energy          fmax
BFGS:    0 19:42:05       -0.320337        1.158029
BFGS:    1 19:42:05       -0.361161        1.312428
BFGS:    2 19:42:05       -0.921519        2.041011
BFGS:    3 19:42:05       55.298954      974.487533
BFGS:    4 19:42:05       -0.923220        2.028761
BFGS:    5 19:42:05       -0.924897        2.016337
BFGS:    6 19:42:05       -0.752559        8.517609
BFGS:    7 19:42:05       -0.970548        1.472894
BFGS:    8 19:42:05       -0.990140        0.939655
BFGS:    9 19:42:05       -0.997803        0.528593
BFGS:   10 19:42:05       -0.999924        0.092418
BFGS:   11 19:42:05       -1.000000        0.007274
BFGS:   12 19:42:05       -1.000000        0.000113
-0.9999999998888873


### Exercise: Trainable Lennard Jones Potential

What we want to do now is to make the parameters `sigma` and `epsilon` learnable from 
data. This will enable us to fit the Lennard Jones potential as best as possible to 
a given dataset. 

To do this we will change a few things from our `LennardJonesTorch` class, 
we will make another called `LennardJonesModule` with these changes.

1. `LennardJonesModule` should *inherit* from `torch.nn.Module`. 
2. The parameters should be set as `torch.nn.Parameter` instances. 

In [100]:
import torch

class LennardJonesModule(torch.nn.Module):

    def __init__(self, sigma=1.0, epsilon=1.0):
        super().__init__()
        self.sigma = torch.nn.Parameter(torch.tensor(sigma))
        self.epsilon = torch.nn.Parameter(torch.tensor(epsilon))

    def atoms_to_tensor(self, atoms):
        return torch.tensor(atoms.positions, requires_grad=True)
    
    def get_lj_energy(self, r):
        # Calculate the Lennard-Jones energy using the class attributes 
        # self.sigma and self.epsilon.
        r6 = r**6
        r12 = r6**2
        return 4 * self.epsilon * (self.sigma/r12 - self.sigma/r6)
    
    def get_forces(self, energy, positions):
        return -torch.autograd.grad(energy, positions, retain_graph=True)[0]

    def calculate(self, atoms):
        # Convert the positions to a tensor
        positions = self.atoms_to_tensor(atoms)
        return self.forward(positions)

    def forward(self, positions):
        # Calculate the pairwise distances
        d = torch.pdist(positions)

        # Calculate the Lennard-Jones energy
        E = torch.sum(self.get_lj_energy(d))
        F = self.get_forces(E, positions)

        # Return the energy and forces
        E = E.detach().numpy() # Convert back to numpy
        F = F.detach().numpy() 
        return E, F

Part of the functionality that we get from inheriting from `torch.nn.Module` is 
the easy access to all the parameters of our class - one way is shown below;

In [102]:
ljm = LennardJonesModule()

for name, parameter in ljm.named_parameters():
    print(name, parameter.item())

sigma 1.0
epsilon 1.0


In [ ]:
import torch
from ase.calculators.lj import LennardJones

def get_lennard_jones_dataset(n=25):
    epsilon = np.random.normal(loc=1.0, scale=0.25)
    sigma = np.random.normal(loc=1.0, scale=0.25)
    
    r0 = 2**(1/6) * sigma

    R = np.linspace(0.8, 2.0, n)
    data = []
    for r in R: 
        E = 4 * epsilon * ((sigma/r)**12 - (sigma/r)**6)
        X = torch.tensor([[0, 0, 0], [r, 0, 0]])
        data.append((X, E))
    return data